In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
cd workspace/theislab/mubind/notebooks/batch

[Errno 2] No such file or directory: 'workspace/theislab/mubind/notebooks/batch'
/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/mubind/notebooks/batch


In [33]:
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '../../annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

Using device: cuda:0


In [100]:
import os

n_sample = 25000
df = []
basedir = '../../../mubind-pipeline/output/snakemake/SELEX'
batch = 0
for tf in os.listdir(basedir):
    for f in os.listdir(os.path.join(basedir, tf)):
        if f.endswith('.tsv.gz'):
            df2 = pd.read_csv(os.path.join(basedir, tf, f), sep='\t', index_col=0) # .head(100)
            print(f, df2.shape)
            assert 'batch' in df2
            # print(df2.columns)
            # df2 = df2.sample(100000)
            n_rounds = len(df2.columns) - 2
            df2.columns =  list(range(n_rounds)) + ['batch', 'is_count_data']
            df2['batch'] = batch
            df2['n_rounds'] = n_rounds
            # df2 = mb.pp.sample_rounds(df2, n_rounds, n_sample_per_round)
            df2 = df2.sample(n_sample)
            print(df2.shape)
            batch += 1
            df.append(df2)
            # assert False
            break



ALX1-ZeroCycle_TACCAA40NTTA_0_0-TACCAA40NTTA_10000.tsv.gz (10000, 7)


ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
# concatenation/reordering
df = pd.concat(df[:1])
df = df[[c for c in df.columns if not c in ['batch', 'is_count_data', 'n_rounds']] + ['batch', 'is_count_data', 'n_rounds']]

In [ ]:
df.shape

In [ ]:
print(df.shape)
dataset = mb.datasets.SelexDataset(df, n_rounds=df['n_rounds'], labels=list(df.columns[:-3]))
train = tdata.DataLoader(dataset=dataset,
                         batch_size=512,
                         # batch_size=512,
                         shuffle=False)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np

In [ ]:
%load_ext line_profiler


In [ ]:
import gc
gc.collect()

In [ ]:
print('total entries', dataset.rounds.shape)

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.expand_frame_repr', False)


In [ ]:
# assign batch and data type

# data['batch'] = 0
# data['is_count_data'] = 1
# n_batches=3

dirichlet_regularization = 0
# for dirichlet_regularization_log in range(-5, 3):

n_epochs = 450
log_each = 25
n_kernels = 2
lr = [0.05] * n_kernels
wd = [0.05,] + [0.005] * (n_kernels - 1)
early_stopping = [5,] + [25] * (n_kernels - 1)
w = 20 # min(len(motif), 6)
opt_kernel_shift = 1
opt_kernel_length = 1

from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 1

import time
t0 = time.time()

criterion = mb.tl.PoissonLoss()

for ignore_kernel in [1]: # [0, 1]:
    print('\n\n##NEW JOB')
    print('ignore kernel', ignore_kernel)
    model_by_k, res_next = mb.tl.optimize_iterative(train, device, w=w, show_logo=0, criterion=criterion,
                                                 opt_kernel_shift=opt_kernel_shift, opt_kernel_length=opt_kernel_length,
                                                 dirichlet_regularization=dirichlet_regularization, # 10 ** dirichlet_regularization_log,
                                                 lr=lr, weight_decay=wd, ignore_kernel=ignore_kernel, use_mono=True,
                                                 num_epochs=n_epochs, early_stopping=early_stopping, use_dinuc=True, # optimiser=torch.optim.LBFGS,
                                                 n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None) #  seed=seed) # seeds.index[0]) #
    print('##DONE....\n\n')


print('total time: %.3f s' % ((time.time() - t0)))
# res = []
# model_by_k = {}

model = model_by_k



### We can visualize the overall results obtained by the network, once the training is finished

In [ ]:
# %load_ext line_profiler
# %lprun -f mb.tl.optimize_simple mb.tl.optimize_iterative(train, device, w=w, show_logo=0, criterion=criterion, opt_kernel_shift=0, opt_kernel_length=0, dirichlet_regularization=dirichlet_regularization, lr=lr, weight_decay=wd, ignore_kernel=ignore_kernel, num_epochs=n_epochs, early_stopping=early_stopping, use_dinuc=False, n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None) #  seed=seed) # seeds.index[0]) #


In [ ]:
import mubind as mb

In [ ]:
model = model_by_k
mb.tl.scores(model, train, k=-1)


In [ ]:
counts = mb.tl.predict(model, train)
counts

In [ ]:
weight_mono_i = model.binding_modes.conv_mono[1].weight
pos_w_sum = weight_mono_i[weight_mono_i > 0].sum()

In [ ]:
weight_mono_i[weight_mono_i > 0]

In [ ]:
rcParams['figure.figsize'] = 5, 5
mb.pl.kmer_enrichment(model, train, log_scale=False, style='scatter', ylab='t1', xlab='p1', k=-1)


In [ ]:
mb.pl.conv(model)
